In [1]:
import copy, torch, torch.nn as nn, torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import snntorch as snn
from snntorch import spikegen
from tqdm import tqdm
import numpy as np

**parameters**

In [2]:
num_in, num_hid, num_out = 784, 256, 10
beta, T, bs, epochs, lr = 1, 50, 128, 3, 1e-3

**device**

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)


**arch network**

In [4]:
device

device(type='cpu')

In [5]:

class SNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1  = nn.Linear(num_in , num_hid, bias=False)
        self.lif1 = snn.Leaky(beta=beta,threshold=0.8,reset_mechanism="zero")
        self.fc2  = nn.Linear(num_hid, num_out, bias=False)
        self.lif2 = snn.Leaky(beta=beta,threshold=0.8,reset_mechanism="zero")
    def forward(self, x):
        x = x.to(self.fc1.weight.dtype)
        mem1 = self.lif1.init_leaky().to(device=x.device, dtype=x.dtype)
        mem2 = self.lif2.init_leaky().to(device=x.device, dtype=x.dtype)
        out  = []
        for t in range(x.size(0)):
            spk1, mem1 = self.lif1(self.fc1(x[t]), mem1)
            spk1 = spk1.to(dtype=self.fc2.weight.dtype)        #
            spk2, mem2 = self.lif2(self.fc2(spk1), mem2)
            out.append(spk2)
        return torch.stack(out)                                # [T,B,10]

In [6]:
tr = transforms.Compose([transforms.ToTensor()])
train_ds = torchvision.datasets.MNIST("data", True , download=True, transform=tr)
test_ds  = torchvision.datasets.MNIST("data", False, download=True, transform=tr)
train_ld = DataLoader(train_ds, bs, True , num_workers=2, pin_memory=True)
test_ld  = DataLoader(test_ds , bs, False, num_workers=2, pin_memory=True)

In [7]:
net = SNN().to(device)
opt = torch.optim.Adam(net.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss().to(device)

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr=0.001)

**train phase**

In [9]:
def train(net,train_ld,epochs):
    for ep in range(1, epochs + 1):
        net.train()
        for imgs, lbls in tqdm(train_ld, desc=f"train {ep}/{epochs}"):
            imgs = imgs.to(device).view(imgs.size(0), -1) 
            lbls = lbls.to(device)
            spk  = spikegen.rate(imgs, T).to(device)
            loss = loss_fn(net(spk).sum(0), lbls)
            opt.zero_grad(); loss.backward(); opt.step()

In [ ]:
# train(net,train_ld,epochs)

train 3/3: 100%|██████████| 469/469 [01:54<00:00,  4.09it/s]


**test phase**

In [10]:
def test(model,loader):
    @torch.no_grad()
    def evaluate(model, loader, dtype):
        model.eval()
        correct = total = 0
        for imgs, lbls in loader:
            imgs = (imgs.to(device).view(imgs.size(0), -1) ).to(dtype)
            spk  = spikegen.rate(imgs, T).to(dtype).to(device)
            preds = model(spk).sum(0).argmax(1)
            correct += (preds == lbls.to(device)).sum().item()
            total   += lbls.size(0)
        return 100 * correct / total


    acc32 = evaluate(net, test_ld, torch.float32)


    net_fp16 = SNN().to(device)
    net_fp16.load_state_dict(net.state_dict())
    net_fp16.half().eval()
    acc16 = evaluate(net_fp16, test_ld, torch.float16)

    print(f"Accuracy FP32 : {acc32:.2f}%")
    print(f"Accuracy FP16 : {acc16:.2f}%")

create a test data spike train for cpp program

In [ ]:
# imgs, lbls = next(iter(test_ld))

# img = imgs[1].to(device)
# label = lbls[1].item()

# img_flat = img.view(-1)


# spk = spikegen.rate(img_flat.unsqueeze(0), T).squeeze(1) 

# print("Label:", label)
# print("Spike array shape:", spk.shape)  # [T, 784]
# print("Spike array (as int):")
# print(spk.int())

# spk_t = spk.int().cpu().tolist()  

# with open("C:\\Users\\Lenovo\\Desktop\\shayan\\QNN\\HW\\FP32\\spikes.cpp", "w") as f:
#     f.write("#include <vector>\n")
#     f.write("#include \"spikes.h\"\n")
#     f.write(f"int label = {label};\n")
#     f.write("std::vector<std::vector<int>> spike_input = {\n")
    
#     for i, row in enumerate(spk_t): 
#         row_str = ", ".join(str(x) for x in row)
#         comma = "," if i < len(spk_t) - 1 else ""
#         f.write(f"    {{{row_str}}}{comma}\n")
#     f.write("};\n")


Label: 2
Spike array shape: torch.Size([50, 784])
Spike array (as int):
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)


create dataset for test with cpp

In [12]:
# num_samples = 1000  

# all_spikes = []
# all_labels = []

# for batch_imgs, batch_lbls in test_ld:
#     for i in range(len(batch_imgs)):
#         if len(all_labels) >= num_samples:
#             break

#         img = batch_imgs[i].to(device)
#         label = batch_lbls[i].item()
#         img_flat = img.view(-1)

#         spk = spikegen.rate(img_flat.unsqueeze(0), T).squeeze(1)  
#         spk_np = spk.int().cpu().tolist()  

#         all_spikes.append(spk_np)         
#         all_labels.append(label)          

#     if len(all_labels) >= num_samples:
#         break

In [13]:


#     # with open("C:\\Users\\Lenovo\\Desktop\\QNN\\HW\\FP32\\TestDataset.cpp", "w") as f:
# #     f.write("#include <vector>\n")
# #     f.write("#include \"TestDataset.h\"\n")
# #     f.write("std::vector<int> labels = {\n")
# #     f.write("    " + ", ".join(str(lbl) for lbl in all_labels) + "\n};\n\n")

# #     f.write("std::vector<std::vector<std::vector<int>>> dataset_spikes = {\n")
# #     for i, sample in enumerate(all_spikes):
# #         f.write("    {\n")
# #         for j, timestep in enumerate(sample):
# #             row_str = ", ".join(str(x) for x in timestep)
# #             comma = "," if j < len(sample) - 1 else ""
# #             f.write(f"        {{{row_str}}}{comma}\n")
# #         comma_sample = "," if i < len(all_spikes) - 1 else ""
# #         f.write(f"    }}{comma_sample}\n")
# #     f.write("};\n")



datatest with txt

In [13]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
T = 50
num_samples = 10000

label_file = open("C:\\Users\\Lenovo\\Desktop\\shayan\\QNN\\posit\\labels.txt", "w")
spike_file = open("C:\\Users\\Lenovo\\Desktop\\shayan\\QNN\\posit\\spikes.txt", "w")

with torch.no_grad():
    count = 0
    for batch_imgs, batch_lbls in test_ld:
        for i in range(len(batch_imgs)):
            if count >= num_samples:
                break

            img = batch_imgs[i].to(device)
            label = batch_lbls[i].item()

            img_flat = img.view(-1).unsqueeze(0)                 # [1, 784]
            spk = spikegen.rate(img_flat, T)                     # [T, 1, 784]
            spk = spk.squeeze(1).int().cpu()                     # [T, 784]

            if spk.shape != (T, 784):
                raise ValueError(f"Expected shape [50, 784], got {spk.shape}")

            spike_file.write(f"# sample {count}\n")
            for timestep in spk:
                line = " ".join(str(bit.item()) for bit in timestep)
                spike_file.write(line + "\n")

            label_file.write(f"{label}\n")
            count += 1

        if count >= num_samples:
            break

spike_file.close()
label_file.close()


In [11]:
net.load_state_dict(torch.load('model_weights.pth', map_location=device))

<All keys matched successfully>

In [12]:
test(net,test_ld)

Accuracy FP32 : 95.25%
Accuracy FP16 : 95.22%


In [14]:
torch.set_printoptions(precision=30)

In [18]:
net_fp16 = SNN()

In [ ]:
net_fp16.load_state_dict(net.state_dict())
net_fp16 = net_fp16.half()

In [17]:
net.fc1.weight

Parameter containing:
tensor([[-0.000267386436462402343750000000,  0.019158702343702316284179687500,
         -0.029394470155239105224609375000,  ...,
          0.021935582160949707031250000000,  0.003732603043317794799804687500,
          0.002060331404209136962890625000],
        [-0.019777085632085800170898437500, -0.014981266111135482788085937500,
         -0.010442411527037620544433593750,  ...,
         -0.020264618098735809326171875000, -0.005962418392300605773925781250,
         -0.029945783317089080810546875000],
        [-0.020148508250713348388671875000,  0.014898568391799926757812500000,
         -0.033275961875915527343750000000,  ...,
         -0.020343888550996780395507812500,  0.001181673258543014526367187500,
          0.008027642965316772460937500000],
        ...,
        [-0.003743059933185577392578125000, -0.022719830274581909179687500000,
          0.033580925315618515014648437500,  ...,
         -0.023739803582429885864257812500,  0.020168386399745941162109375000

In [20]:
net_fp16.fc1.weight

Parameter containing:
tensor([[-0.000267505645751953125000000000,  0.019165039062500000000000000000,
         -0.029388427734375000000000000000,  ...,
          0.021942138671875000000000000000,  0.003732681274414062500000000000,
          0.002059936523437500000000000000],
        [-0.019775390625000000000000000000, -0.014984130859375000000000000000,
         -0.010444641113281250000000000000,  ...,
         -0.020263671875000000000000000000, -0.005962371826171875000000000000,
         -0.029953002929687500000000000000],
        [-0.020141601562500000000000000000,  0.014900207519531250000000000000,
         -0.033264160156250000000000000000,  ...,
         -0.020339965820312500000000000000,  0.001181602478027343750000000000,
          0.008026123046875000000000000000],
        ...,
        [-0.003742218017578125000000000000, -0.022720336914062500000000000000,
          0.033569335937500000000000000000,  ...,
         -0.023742675781250000000000000000,  0.020172119140625000000000000000

In [ ]:
# torch.save(net.state_dict(),'model_weights.pth')

Write weights with format cpp in weights.cpp

with 6 digits

In [ ]:
# net.eval()
# with open("C:\\Users\\Lenovo\\Desktop\\shayan\\QNN\\HW\\FP32\\weightst.cpp", "w") as cpp_file:
#     cpp_file.write("#include <vector>\n\n")
#     cpp_file.write("#include \"weights.h\"\n\n")

#     for name, param in net.state_dict().items():
#         cpp_name = name.replace(".", "_")
#         data = param.cpu().numpy()
#         shape = data.shape

#         if "weight" in name and len(shape) == 2:
#             data = data.T  # Transpose to [input][output]
#             shape = data.shape

#             cpp_file.write(f"// shape: {shape} // Transposed\n")
#             cpp_file.write(f"std::vector<std::vector<float>> {cpp_name} = {{\n")

#             for row in data:
#                 row_str = ", ".join(f"{v:.6f}" for v in row)
#                 cpp_file.write(f"    {{{row_str}}},\n")

#             cpp_file.write("};\n\n")

#         elif "bias" in name and len(shape) == 1:
#             cpp_file.write(f"// shape: {shape}\n")
#             cpp_file.write(f"std::vector<float> {cpp_name} = {{\n    ")
#             cpp_file.write(", ".join(f"{v:.6f}" for v in data))
#             cpp_file.write("\n};\n\n")


with 25 digits

In [18]:
net.eval()
with open("C:\\Users\\Lenovo\\Desktop\\shayan\\QNN\\posit\\weights.cpp", "w") as cpp_file:
    cpp_file.write("#include <vector>\n\n")
    cpp_file.write("#include \"weights.h\"\n\n")

    for name, param in net.state_dict().items():
        cpp_name = name.replace(".", "_")
        data = param.cpu().numpy()
        shape = data.shape

        if "weight" in name and len(shape) == 2:
            data = data.T  # Transpose to [input][output]
            shape = data.shape

            cpp_file.write(f"// shape: {shape} // Transposed\n")
            cpp_file.write(f"const std::vector<std::vector<float>> {cpp_name} = {{\n")

            for row in data:
                row_str = ", ".join(f"{v:.25f}" for v in row)  
                cpp_file.write(f"    {{{row_str}}},\n")

            cpp_file.write("};\n\n")

        elif "bias" in name and len(shape) == 1:
            cpp_file.write(f"// shape: {shape}\n")
            cpp_file.write(f"const std::vector<float> {cpp_name} = {{\n    ")
            cpp_file.write(", ".join(f"{v:.25f}" for v in data)) 
            cpp_file.write("\n};\n\n")


***julia format for weights**

In [ ]:

def save_weights_to_julia(state_dict, filename="weights.jl"):

    with open(filename, "w") as jl_file:
        jl_file.write("# This file contains model weights exported from PyTorch to Julia format.\n\n")

        for name, param in state_dict.items():
            julia_name = name.replace(".", "_")
            data = param.cpu().numpy()
            shape = data.shape

            if "weight" in name and len(shape) == 2:
                # Transpose the weight matrix for compatibility.
                data = data.T
                shape = data.shape

                jl_file.write(f"# shape: {shape} # Transposed\n")
                jl_file.write(f"const {julia_name} = [\n")

                for i, row in enumerate(data):
                    row_str = ", ".join(f"{v:.25f}" for v in row)
                    separator = ";\n" if i < len(data) - 1 else "\n"
                    jl_file.write(f"    {row_str}{separator}")

                jl_file.write("]\n\n")
net.eval()

save_weights_to_julia(net.state_dict())

print("Weights have been saved to weights.jl")
